In [1]:
import pandas as pd
import openpyxl as px
from openpyxl.styles import Alignment, Font
import datetime
import calendar
from dateutil.relativedelta import relativedelta
import os
import tkinter as tk
from tkinter import messagebox




def invoice(invoice_month):
    #######################
    # 日付の処理
    #######################

    #invoice_month = input("請求書を作成する年月を入れてください(例：2110)")
    #入力された値をdataに変換
    dte = datetime.datetime.strptime(invoice_month, "%y%m")


    # 日本語でstrftimeが表示できないエラーの回避
    import locale
    locale.setlocale(locale.LC_CTYPE, "Japanese_Japan.932")

    # 選択月の末日を習得
    EOM = dte.replace(day=calendar.monthrange(dte.year, dte.month)[1])
    # print(EOM.strftime("%Y年%m月%d日"))

    # 翌月末を取得
    next_month = EOM + relativedelta(months=1)


    ####################
    # Excelを読み込み
    ####################

    # 請求書一覧から顧客CDをDFに取得
    customer_df = pd.read_excel("請求一覧創造太郎.xlsx", sheet_name="顧客管理テーブル", index=0, header=1)

    # inputからその月のシートを取得
    sheetname = "請求一覧" + invoice_month
    month_df = pd.read_excel("請求一覧創造太郎.xlsx", sheet_name=sheetname, header=2, index=False)

    # 選択月のシートに顧客名を追加
    month_df = month_df.merge(customer_df, left_on="相手先コード", right_on="顧客ＣＤ", how="left")

    # 選択月の相手先コードのユニークを作成
    sup_cd = month_df["相手先コード"].unique()
    sup_cd.sort()



    #######################
    # 請求書Excelを作成
    #######################

    #----------- 出力セルの定義 ----------------
    date_loc = "H1"                # 日付
    ivno_loc = "H2"                # 請求書No.
    Co_loc = "A6"                  # 会社名
    sum_loc = "G28"                # 合計金額
    dl_loc = "B35"                 # 振込期限
    #------------------------------------------

    # 出力先のフォルダを作成 (出力先ファイルを指定できるようにしてもいい)
    n_path = os.getcwd()
    folder_name = EOM.strftime("%Y年%m月") + "分_請求書"
    path = n_path + "/" + folder_name
    os.makedirs(path, exist_ok=True)

    # 顧客コード毎に請求書を作成
    x = 1
    for customer_cd in sup_cd:
        string = str(x).zfill(2) # 請求書No.用の連番
        
        wb = px.load_workbook("請求書創造太郎.xlsx")
        ws = wb["請求書"]
        
        df = month_df[month_df["相手先コード"] == customer_cd]

        bold = Font(bold=True)

        #------- 月一覧のDFからカラムごとにリスト化 --------------
        odr_no = df["受注No."].to_list()
        cus_cd = df["相手先コード"].to_list()
        cus_name = df["顧客名"].to_list()
        product_name1 = df["商品名１"].to_list()
        product_name2 = df["商品名２"].to_list()
        en = df["金額（税抜き）"].to_list()
        bikou = df["備考"].to_list()
        #-------------------------------------------------------

        # エクセルにデータを入力
        ws[date_loc] = EOM.strftime("%Y年%m月%d日")                                 # 日付
        ws[ivno_loc] = invoice_month + string                                       # 請求書No.
        ws[Co_loc] = cus_name[0] + "  御中"                                         # 会社名
        ws[dl_loc] = "お振込み期限　　：　　" + next_month.strftime("%Y年%m月%d日")   # 振込期限

        j = 0
        for i in range(18,27,2):
            if j == len(df):
                break
            # 商品名２が空白の時
            if pd.isna(product_name2[j]):
                ws.unmerge_cells(start_row=i, start_column=2, end_row=i, end_column=4)      #結合解除
                ws.unmerge_cells(start_row=i+1, start_column=2, end_row=i+1, end_column=4)  #結合解除
                ws.merge_cells(start_row= i, start_column=2, end_row= i+1, end_column=4)    #結合
                ws.cell(row= i, column=2, value = product_name1[j]).alignment = Alignment(horizontal="center", vertical="center")    # 商品名１
            else:
                ws.cell(row= i, column=2).value = product_name1[j]    # 商品名１
                ws.cell(row= i+1, column=2).value = product_name2[j]  # 商品名２
            
            ws.cell(row= i, column=1).value = odr_no[j]           # 受注No.
            ws.cell(row= i, column=5).value = 1                   # 数量
            ws.cell(row= i, column=6).value = "式"                # 式
            ws.cell(row= i, column=7).value = en[j]               # 金額
            ws.cell(row= i, column=8).value = bikou[j]            # 備考
            j += 1

        total = df["金額（税抜き）"].sum()
        tax = total * 0.1

        ws.cell(row= 18+len(df)*2+1 , column=2, value = "以上に掛かる消費税").alignment = Alignment(horizontal="center", vertical="bottom")     # 以上に掛かる消費税
        ws.cell(row= 18+len(df)*2+2 , column=2, value = "～　以　下　余　白　～").alignment = Alignment(horizontal="center")                    # ～以下余白～
        ws.cell(row= 18+len(df)*2+2 , column=2).font = bold                                                                                    # 以下余白を太字に
        ws.cell(row= 18+len(df)*2 , column=7, value = tax).alignment = Alignment(horizontal="center", vertical="bottom")                       # 消費税
        ws[sum_loc] = total + tax                                                                                                              # 合計金額

        # ファイルの書き出し
        os.chdir(path)
        wb.save(EOM.strftime("%Y年%m月") + "分請求書_【" + cus_name[0] + "様】.xlsx")
        os.chdir(n_path)
        x += 1

    now_time = datetime.datetime.now()
    now_time = now_time.strftime("%Y/%m/%d %H:%M:%S")

    #print("請求書を作成しました。" + now_time )

now_time = datetime.datetime.now()
tx_in = now_time.strftime("%y%m")

# ------ tkinter ------------------------
def create_btm():
    invoice_month = txt.get()
    print(invoice_month)
    print(type(invoice_month))
    invoice(invoice_month)
    messagebox.showinfo('請求書作成', '請求書を作成しました。')
    
def end_btn():
    root.destroy()
    

if __name__ == "__main__":
    global root
    root = tk.Tk()
    root.title("請求書作成")
    root.geometry("300x200+840+440")
    # ラベル
    lbl = tk.Label(text='年月')#ラベル名
    lbl.place(x=80, y=70)   # 表示位置 
    # lbl.pack()
    # テキストボックス
    txt = tk.Entry(width=10)#幅３０
    txt.place(x=120, y=70)    # 表示位置
    txt.insert(tk.END,tx_in) # プレースホルダー(初期値)
    # txt.pack()

    btn = tk.Button(root, text='作成', command=create_btm)
    btn.place(x=110, y=120) #表示位置
    
    btn = tk.Button(root, text='終了', command=end_btn)
    btn.place(x=150, y=120) #表示位置

    root.mainloop()
# ---------------------------------------